### Imports

In [ ]:
import json
import math
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets

from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap

from qiskit.opflow import X, Y, Z, I, CircuitStateFn
from qiskit.opflow.state_fns import StateFn
from qiskit.opflow.expectations import PauliExpectation, MatrixExpectation
from qiskit.opflow.converters import CircuitSampler
from qiskit.providers.aer import QasmSimulator


### Definizione dei dati

Dati sintetici per testare l'esecuzione del codice

In [ ]:
DATA = np.array([[0.8, 0.8, 0.8, 0.8], [1, 1, 1, 1]])
FEATURES = DATA.shape[1]
SIZE = DATA.shape[0]

### Circuito enconding

Viene utilizzata una ZZFeatureMap

In [ ]:
encode_map = ZZFeatureMap(feature_dimension=FEATURES, reps=1, entanglement='linear', insert_barriers=True)
encode_circuit = encode_map.bind_parameters(DATA[1])
encode_circuit.decompose().draw(output='mpl')

## EM Distance


Dati due stati $|\psi>$ e $|\phi>$, si definisce la distanza EM approssimata come: 


$
\widetilde{D}_{EM} = max_{H_i}| <\phi|H| \phi> - <\psi|H| \psi>|  :  ||H||_L \leq 1
$

dato $O_n$ insieme degli observbles su n qbit, si definisce $H$

$
H \in O_n : ||H||_L = 2 max_{i = 1,\dots,n} (min\{{||H- H_i ||_{\inf} : H_i \in O_n \text{  identità sull'iesimo qbit}}\})
$

Il valore $<\phi|H| \phi>$ equivale al calcolo dell'expectation dell'osservable H rispetto allo stato $|\phi>$


### Esecuzione passo passo

generate_pauli(n): Genera le combinazioni di operatori pauli (X,Y,Z) sui singoli qbit per 
generare un subset di obsevables con constante di lipshitz minore o uguale a 1

Nello specifico, gli observables presi in considerazione sono:

- Pauli Z per ogni qbit
- Per ogni qbit k:
    - Per ogni qbit j < k applico Pauli X
    - Per il qbit k applico Pauli X se PARI e Pauli Y se DISPARI
    - Per ogni qbit k < j applico Pauli Z

Vegono considerate quindi n+1 osservabili. Ad esempio, per n=4 qbit verrano generati i gate:
ZZZZ, YZZZ, XXZZ, XXYZ, XXXX



In [ ]:
def even(x):
    return (x % 2) == 0


def generate_pauli(n):
    hamiltonians = []

    h = 1
    for _ in range(n):
        h = h^Z
    hamiltonians.append(h)

    for k in range(n):
       
        h = 1

        if(k != 0):
            for _ in range(k):
                h = h^X

        if even(k+1):
            h = h^X
        else:
            h = h^Y

        if (k+1 != n):
            for _ in range(k+1, n):
                h = h^Z


        hamiltonians.append(h)

    return hamiltonians

Compongo il circuito per i due vettori da confrontare.

Dati due vettori x,y viene applicata una ZZFeature map per l'encoding rispettivamente nei stati $|\psi>$ e $|\phi>$.

Viene successivamente composto il circuito che calcola:
$<\phi|H| \phi>$ e $<\psi|H| \psi>$

con H matrice generata dalla funzione generate_pauli


In [ ]:
x = CircuitStateFn(encode_map.bind_parameters(DATA[0]))
y = CircuitStateFn(encode_map.bind_parameters(DATA[1]))
n = x.num_qubits

observables = generate_pauli(n)

circuit = x.adjoint().compose(observables[4]).compose(x)

print(circuit)

Calcolo dell'expected value, questo viene calcolato per ogni obsevable e per entrambi i vettori, viene calcolata quindi la differenza in valore assoluto e presa la maggiore, seguendo la definizione di distanza EM

In [ ]:
max = 0

for h in observables:

    print(f"Obeservable [{h}]")
    circuit_x = x.adjoint().compose(h).compose(x)
    circuit_y = y.adjoint().compose(h).compose(y)

    EM = abs(circuit_x.eval().real - circuit_y.eval().real)

    print(f'''EM Distance
       |E(x) - E(y)| = {abs(circuit_x.eval().real - circuit_y.eval().real)}''')
        
    print('\n')

print(f"Final EM: {max}")


### EM Distance function
Calcolo della distanza con data enconding usando una ZZFeature map

In [ ]:
def em_distance_approximate(data_1, data_2, encode_map, observables, simulator):

    x = CircuitStateFn(encode_map.bind_parameters(data_1))
    y = CircuitStateFn(encode_map.bind_parameters(data_2))

    em = 0

    for h in observables:

        measurable_x = StateFn(h, is_measurement=True).compose(x)
        measurable_y = StateFn(h, is_measurement=True).compose(y)

        expectation_x = PauliExpectation().convert(measurable_x)
        expectation_y = PauliExpectation().convert(measurable_y)

        sampler_x = CircuitSampler(simulator).convert(expectation_x)
        sampler_y = CircuitSampler(simulator).convert(expectation_y)

        current_expectation = abs(sampler_x.eval().real - sampler_y.eval().real)

        if current_expectation > em:
            em = current_expectation

    return em

def em_distance_exact(data_1, data_2, encode_map, observables):

    x = CircuitStateFn(encode_map.bind_parameters(data_1))
    y = CircuitStateFn(encode_map.bind_parameters(data_2))

    em = 0

    for h in observables:

        circuit_x = x.adjoint().compose(h).compose(x)
        circuit_y = y.adjoint().compose(h).compose(y)

        current_expectation = abs(circuit_x.eval().real - circuit_y.eval().real)

        if current_expectation > em:
            em = current_expectation

    return em


def em_dissimilarity_matrix(dataset, approximate = False):

    encode_map = ZZFeatureMap(feature_dimension=dataset.shape[1], reps=1, entanglement='linear', insert_barriers=True)
    n = encode_map.num_qubits
    observables = generate_pauli(n)
    mat = np.zeros((dataset.shape[0], dataset.shape[0]))

    if approximate:
        simulator = QasmSimulator()

        for i in range(dataset.shape[0]):
            for j in range(i):
                dis = em_distance_approximate(dataset[i], dataset[j], encode_map, observables, simulator)
                mat[i,j] = dis
                mat[j,i] = dis
    else:

        for i in range(dataset.shape[0]):
            for j in range(i):
                dis = em_distance_exact(dataset[i], dataset[j], encode_map, observables)
                mat[i,j] = dis
                mat[j,i] = dis

    return mat

def euclidean_matrix(dataset):
    mat = np.zeros((dataset.shape[0], dataset.shape[0]))

    for i in range(dataset.shape[0]):
        for j in range(i):
            dis = np.sqrt(np.sum((dataset[i] - dataset[j])**2))
            mat[i,j] = dis
            mat[j,i] = dis

    return mat


def manhattan_matrix(dataset):
    mat = np.zeros((dataset.shape[0], dataset.shape[0]))

    for i in range(dataset.shape[0]):
        for j in range(i):
            dis = np.sum(np.abs(dataset[i] - dataset[j]))
            mat[i,j] = dis
            mat[j,i] = dis

    return mat

In [ ]:
iris = datasets.load_iris()['data'][15:35, :]
em_matrix = em_dissimilarity_matrix(iris, approximate=False)

In [ ]:
%matplotlib inline

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15,6)

axs[0].matshow(em_matrix, cmap='RdPu')
axs[0].set_title("EM Distance")
axs[1].matshow(manhattan_matrix(iris), cmap='RdPu')
axs[1].set_title("Manhattan Distance")
axs[2].matshow(euclidean_matrix(iris), cmap='RdPu')
axs[2].set_title("Euclidean Distance")

fig.show()

In [ ]:
blobs, classes = datasets.make_blobs(n_samples=50, random_state=10, n_features=2, centers=2)
blobs = np.array(blobs)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(blobs[:,0], blobs[:,1], c=classes, cmap="Accent")

In [ ]:
em_matrix = em_dissimilarity_matrix(blobs)

In [ ]:
%matplotlib inline

fig, axs = plt.subplots(1,3)
fig.set_size_inches(15,6)

axs[0].matshow(em_matrix, cmap='RdPu')
axs[0].set_title("EM Distance")
axs[1].matshow(manhattan_matrix(blobs), cmap='RdPu')
axs[1].set_title("Manhattan Distance")
axs[2].matshow(euclidean_matrix(blobs), cmap='RdPu')
axs[2].set_title("Euclidean Distance")

fig.show()